In [ ]:
import numpy as np
def cpca(X,d_): #X: list of (d_v,n)
    X = np.concatenate(X)
    X = X-X.mean(axis=1).reshape(-1,1)
    cov_X = X.dot(X.T)
    eigenval,eigenvec = np.linalg.eig(cov_X)
    eigenval = np.real(eigenval)
    eigenvec = np.real(eigenvec)    
    idx = np.argpartition(eigenval,-d_)[-d_:]
    select_vec = eigenvec[:,idx]
    out = select_vec.T.dot(X)
    return out

def dpca(X,d_):
    dim_reductions =[]
    for x in X:
        x = x-x.mean(axis=1).reshape(-1,1)
        cov_x = x.dot(x.T)
        eigenval,eigenvec = np.linalg.eig(cov_x)
        eigenval = np.real(eigenval)
        eigenvec = np.real(eigenvec)    
        idx = np.argpartition(eigenval,-d_)[-d_:]
        select_vec = eigenvec[:,idx]
        out = select_vec.T.dot(x)
        dim_reductions.append(out)
    return sum(dim_reductions)/len(dim_reductions)


a = np.array([[1,2],[3,4]])
b = np.random.rand(3,4)
# np.concatenate([a,b])
a-a.mean(1)
b-b.mean(1).reshape(-1,1)
np.linalg.eig(b.dot(b.T))



In [ ]:
import scipy.io as sio
import numpy as np
from sklearn.cluster import KMeans
def read_data(filename):
    data = sio.loadmat(filename)
    X = data['X']
    labels = data['Y']
    return X,labels


X,labels=read_data('datasets/data sets/MSRC-v1.mat')
print(X.shape)
X = [X[0,i].T for  i  in range(X.shape[1])]
cpca(X,10)
dimreduce_cpca =  cpca(X,200)
kmeans= KMeans(random_state=0,n_clusters=7).fit(dimreduce_cpca.T)
print(kmeans.labels_.shape,labels.squeeze().shape)

# ACCuracy (ACC), 
# #Normalized Mutual Iinformation (NMI), 
# Purity, F-score and Adjusted Rand Index(ARI)
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import accuracy_score,adjusted_rand_score

nmi = normalized_mutual_info_score(labels.squeeze(),kmeans.labels_)
print(f'Normalized Mutual Information is {nmi}')

In [ ]:
import tarfile
import os
from tqdm import tqdm
import numpy as np



def untar(fname, dirs):
    """
    解压tar.gz文件
    :param fname: 压缩文件名
    :param dirs: 解压后的存放路径
    :return: bool
    """
    try:
        t = tarfile.open(fname)
        t.extractall(path = dirs)
        return True
    except Exception as e:
        print(e)
        return False

# untar('datasets/msrcorid.tar.gz','./datasets/')
from PIL import Image
def read_imgs():
    path = 'datasets/msrcorid'
    filepaths = []
    img_arrays = []
    labels = []
    for root, dirs, files in tqdm(os.walk(path)):
        # print(dirnames)     
        for file in files:
            if(file.endswith('JPG')):                         
                file_path = os.path.join(root, file)
                img = np.asarray(Image.open(file_path))
                img_arrays.append(img.transpose((2,0,1)).reshape((3,-1,1)))
                labels.append(file[:3])   
                filepaths.append(file_path)           
       
    return img_arrays,labels
def color_moment(n,img_tensor):
    color_mean = img_tensor.mean(axis=1)
    temp= ((img_tensor-np.expand_dims(color_mean,axis = 1))**n).mean(axis=1)
    return temp**(1/n)

img_arrays,labels = read_imgs() #img_arrays = [(ch,n_pixel,1),...]
img_tensor = np.concatenate(img_arrays,axis = 2)
color_mean = img_tensor.mean(axis=1)
# color_moment(2,img_tensor)

